*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import string
import nltk

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
#Solution inspired by : https://medium.com/analytics-vidhya/how-to-build-a-simple-sms-spam-filter-with-python-ee777240fc

train_data = pd.read_csv(train_file_path, sep='\t', header=None, names=["label", "sms"])
train_data.head()

In [ ]:
test_data = pd.read_csv(test_file_path, sep='\t', header=None, names=["labels", "sms"])
test_data.head()

In [ ]:
#We get stop words and punctuation
nltk.download('stopwords')
nltk.download('punkt')

stopwords = nltk.corpus.stopwords.words('english')
punctuation = string.punctuation

print(type(stopwords[:5]))
print(punctuation)

In [ ]:
def pre_process(sms):
  remove_punct = "".join([word.lower() for word in sms if word not in punctuation])
  tokenize = nltk.tokenize.word_tokenize(remove_punct)
  remove_stopwords = [word for word in tokenize if word not in stopwords]
  return remove_stopwords

train_data["processed"] = train_data["sms"].apply(lambda x:pre_process(x))
train_data.head()



In [ ]:
def categorize_words():
  spam_words = []
  ham_words = []

  for sms in train_data["processed"][train_data["label"] == "spam"]:
    for word in sms:
      spam_words.append(word)
  for sms in train_data["processed"][train_data["label"] == "ham"]:
    for word in sms:
      ham_words.append(word)

  return spam_words, ham_words

spam_words, ham_words = categorize_words()

print(spam_words[:])
print(ham_words[:])


print(spam_words.count("urgent"))
print(ham_words.count("urgent"))


In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  spam_counter = 0
  ham_counter = 0
  prediction = []

  for word in pred_text.split():
    spam_counter += spam_words.count(word)
    ham_counter += ham_words.count(word)

  prediction.append(round((ham_counter / (ham_counter + spam_counter) * 100)))

  if ham_counter > spam_counter:
    prediction.append("ham")
  else:
    prediction.append("spam")

  print(prediction)
    
  return (prediction)


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):

    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
